In [1]:
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import requests
import random
from sqlalchemy import create_engine
from config import trefle_key
token = trefle_key
# Document Resource :https://docs.trefle.io/docs/examples/snippets

In [13]:
#set-up to generate a list of returned data in a loop
search_input = 'lady slipper'
search = search_input.replace(" ","%20")

name_url = f'https://trefle.io/api/v1/plants/search?token={token}&q={search}'
results = requests.get(name_url)
meta_data = results.json()['meta']
meta_total = meta_data["total"]
if meta_total/20 < 1:
    pages = 1
elif meta_total % 20 != 0:
    pages = round(meta_total/20 +1,0)
else:
    pages = int(meta_total/20)
    
pages = int(pages)
pages
    

1

### Create empty lists and loop through pages


In [14]:
common_name=[]
scientific_name =[]
image_url=[]
family_common_name =[]


for page in range(1,pages+1):
    page_url = f'https://trefle.io/api/v1/plants/search?token={token}&q={search}&page={page}'
    print(page_url)
    results = requests.get(page_url)
    data = results.json()['data']
    names = [sub['common_name'] for sub in data]
    s_name = [sub['scientific_name'] for sub in data]
    image = [sub['image_url'] for sub in data]
    family = [sub['family_common_name'] for sub in data]
    common_name.append(names)
    scientific_name.append(s_name)
    family_common_name.append(family)
    image_url.append(image)

    i = +1
scientific_name

https://trefle.io/api/v1/plants/search?token=gIpeDhQj51LoZphqppfDxy1OSKQ1P8u9p8p5JUuZB_g&q=lady%20slipper&page=1


[['Cypripedium calceolus',
  'Cypripedium parviflorum',
  'Cypripedium reginae',
  'Cypripedium candidum',
  'Cypripedium guttatum',
  'Cypripedium passerinum',
  'Cypripedium montanum',
  'Cypripedium arietinum',
  'Cypripedium californicum',
  'Cypripedium kentuckiense',
  'Cypripedium japonicum',
  'Cypripedium acaule',
  'Cypripedium fasciculatum',
  'Centrosema virginianum',
  'Cyclanthera pedata',
  'Cypripedium × columbianum',
  'Cypripedium × andrewsii',
  'Cypripedium × herae']]

In [15]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate throu the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [16]:
scientific_names_final = flatten_list(scientific_name)
common_name_final = flatten_list(common_name)
image_url_final = flatten_list(image_url)
family_common_name_final = flatten_list(family_common_name)


In [17]:
# combine into one pandas dataframe
plants_returned = pd.DataFrame({"Common_Name" : common_name_final,
                                "Scientific_Name": scientific_names_final,
                            "Family_Common_Name": family_common_name_final,
                                    "Image_Url" : image_url_final})



In [49]:
plants_returned

,Common_Name,Scientific_Name,Family_Common_Name,Image_Url
0,Lady's-slipper,Cypripedium calceolus,None,https://bs.floristic.org/image/o/c832369ad93cd...
1,Lesser yellow lady's slipper,Cypripedium parviflorum,None,https://bs.floristic.org/image/o/a69bf25c67aea...
2,Showy lady's slipper,Cypripedium reginae,None,https://d2seqvvyy3b8p2.cloudfront.net/b7f2d9f7...
3,White lady's slipper,Cypripedium candidum,None,https://storage.googleapis.com/powop-assets/or...
4,Spotted lady's slipper,Cypripedium guttatum,None,https://d2seqvvyy3b8p2.cloudfront.net/fbe41511...
5,Sparrowegg lady's slipper,Cypripedium passerinum,None,https://d2seqvvyy3b8p2.cloudfront.net/46d0db6e...
6,Mountain lady's slipper,Cypripedium montanum,None,https://d2seqvvyy3b8p2.cloudfront.net/1aee1b17...
7,Ram's head lady's slipper,Cypripedium arietinum,None,https://d2seqvvyy3b8p2.cloudfront.net/8c069578...
8,California lady's slipper,Cypripedium californicum,None,None
9,Kentucky lady's slipper,Cypripedium kentuckiense,None,None


In [50]:
# almost_final_plant_list = plants_returned[plants_returned['Common_Name'] isNull]
  #df_filtered = df[df['Age'] >= 25] 
    
# arg = plants_returned[isNull(plants_returned['Common_Name'])]

# almost_final_plant_list= plants_returned.drop(plants_returned[plants_returned['Common_Name']==None].index,inplace = True)
# index_names = df[ df['Age'] == 21 ].index 
# df.drop(index_names, inplace = True)   
# index_names = plants_returned[plants_returned['Common_Name'] == None].index
# arg1 = plants_returned.drop(index_names,inplace = True)
# arg1
    
#     df.drop(df[df['Age'] < 25].index, inplace = True)
# almost_final_plant_list
# plants_returned
                                          

In [44]:
final_plant_list = plants_returned.replace(to_replace='None', value=np.nan).dropna().reset_index(drop=True)
final_plant_list

,Common_Name,Scientific_Name,Family_Common_Name,Image_Url
0,Spurred butterfly pea,Centrosema virginianum,Pea family,https://bs.floristic.org/image/o/2a73db0405cb3...
1,Slipper goard,Cyclanthera pedata,Cucumber family,https://bs.floristic.org/image/o/2397a7c80a5be...


In [19]:
rows = len(final_plant_list.index)
if rows < 4:
    selected = [i for i in final_plant_list.index]
else:
    selected = [random.randrange(0, rows, 1) for i in range(4)]
selected

[0, 1]

In [11]:
import requests

In [12]:
final_plant_list.loc[1](final_plant_list['Image_Url'])

TypeError: 'Series' object is not callable

In [ ]:
for i in selected:
    image = (f"'{final_plant_list.loc[i](final_plant_image['Image_Url'])}'")
    print(image)
    

In [ ]:
# final_plant_list = plants_returned.drop(plants_returned.loc[plants_returned.Common_Name == 'None'].index, inplace=True)
# final_plant_list

In [ ]:
# Need to get data on all species and species and save it as a DB 
species_url = url = f"https://trefle.io/api/v1/species?token={token}"
all_species = requests.get(species_url)
species_list = all_plants.json()['data']
# species_list

In [ ]:
# Need to get data on all edible plants and species and save it as a DB 
edible_url = f"https://trefle.io/api/v1/plants?filter_not%5Bedible_part%5D=null&token={token}"
all_edible = requests.get(edible_url)
edible_list = all_plants.json()['data']
# edible_list

In [ ]:
# Get all Tallest trees
tall_tree_url = f"https://trefle.io/api/v1/plants?filter_not%5Bmaximum_height_cm%5D=null&filter%5Bligneous_type%5D=tree&order%5Bmaximum_height_cm%5D=desc&token={token}"
all_tall_tree = requests.get(tall_tree_url)
tall_tree_list = all_tall_tree.json()['data']
# tall_tree_list

In [ ]:
# Get all the Countries/zones
zones_url = f"https://trefle.io/api/v1/distributions?token={token}"
zones_list = requests.get(zones_url).json()['data']
zones_list

In [ ]:
requests.get(zones_url).json()

In [ ]:
zone_test = requests.get(zones_url).json()
# zone_test

In [ ]:
# Get all Family
family_url = "https://trefle.io/api/v1/families/?token={token}"
family_list = requests.get(family_url).json()['data']
# family_list

In [ ]:
# Get all genus
genus_url = f"https://trefle.io/api/v1/genus?token={token}"
genus_list = requests.get(genus_url).json()['data']

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.trefle
collection = db.plants

In [ ]:
collection.insert_many(plants_list)

In [ ]:
collection = db.species
collection.insert_many(species_list)

In [ ]:
collection = db.edible_plants
collection.insert_many(edible_list)

In [ ]:
collection = db.tall_tree
collection.insert_many(tall_tree_list)

In [ ]:
collection = db.country_zones
collection.insert_many(zones_list)

In [ ]:
collection = db.genus
collection.insert_many(genus_list)

In [ ]:
collection =db.family
collection.insert_many(family_list)

In [4]:
import math
meta_data = 180
pages = math.ceil(meta_data/20)
pages

9